In [ ]:
%load_ext autoreload
%autoreload 2

In [6]:
from environment import RNAInvEnvironment, make_vec_env, Monitor
from RNA_helper import get_puzzle
import torch as th
from models import EmbeddinsFeatureExtractor
from stable_baselines3.common import logger

In [7]:
# 1, 41, 84, 92, 97, 5, 
puzzle_idx=99
objective_structure, sequence, puzzle_name = get_puzzle(idx=puzzle_idx, return_name=True, verbose=False)
len(objective_structure)

99 0.0 100000.0
no



382

In [14]:
max_steps = 1
features_dim = 512
EMBEDDING_DIM = 16
metric = 'energies_mse'
model_name = puzzle_name.lower().replace(' ', '_') + f'_{features_dim}_{EMBEDDING_DIM}_{metric}'
print(model_name)

teslagon_512_16_energies_mse


In [15]:
env_kwargs = {
    'objective_structure': objective_structure,
    'max_steps': max_steps,
    'tuple_obs_space': True,
    'metric_type': metric,
    'sequences_file': f'solved_puzzles/{model_name}.txt'
}

In [16]:
n_envs=12
env = make_vec_env(RNAInvEnvironment, n_envs=n_envs, env_kwargs=env_kwargs)
# env = RNAInvEnvironment(objective_structure=objective_structure, max_steps=max_steps, tuple_obs_space=True)

/opt/tljh/user/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [17]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.policies import ActorCriticPolicy

In [18]:
policy_kwargs = dict(
    features_extractor_class=EmbeddinsFeatureExtractor,
    features_extractor_kwargs=dict(EMBEDDING_DIM=EMBEDDING_DIM, features_dim=features_dim),
)

In [19]:
model = PPO(
    ActorCriticPolicy,
    env,
    verbose=1,
    tensorboard_log='tensorboard_logs',
    n_steps=512,
    gamma=0.99,
    policy_kwargs=policy_kwargs
)

Using cuda device


In [20]:
# log_path = f"logs/{model_name}"
# # set up logger
# new_logger = logger.configure(log_path, ["stdout", "csv", "log", "tensorboard", "json"])
# model.set_logger(new_logger)

In [21]:
# eval_env = make_vec_env(
#     RNAInvEnvironment, n_envs=1,
#     env_kwargs={'objective_structure': objective_structure, 'max_steps': max_steps, 'tuple_obs_space': True}
# )

eval_env = make_vec_env(
    RNAInvEnvironment, n_envs=1,
    env_kwargs=env_kwargs,
    monitor_dir=f'logs/{model_name}',
    monitor_kwargs={
        'info_keywords': (
            'free_energy',
            'structure_distance',
            'energy_to_objective',
            'energy_reward',
            'distance_reward',
            'folding_struc',
            'sequence',
            'solved',
            'unique_sequences_N'
        )
    }
)

In [22]:
eval_callback = EvalCallback(
    eval_env = eval_env,
    eval_freq=512*5,
    n_eval_episodes=1024,
    deterministic=True,
    verbose=1,
    best_model_save_path=f'models/{model_name}',
)


In [ ]:
%%time
model.learn(
    total_timesteps=1_000_000,
    tb_log_name=model_name,
    callback=[eval_callback]
)

Logging to tensorboard_logs/teslagon_512_16_energies_mse_1
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 1         |
|    ep_rew_mean     | -3.36e+04 |
| time/              |           |
|    fps             | 5         |
|    iterations      | 1         |
|    time_elapsed    | 1050      |
|    total_timesteps | 6144      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -3.24e+04   |
| time/                   |             |
|    fps                  | 5           |
|    iterations           | 2           |
|    time_elapsed         | 2272        |
|    total_timesteps      | 12288       |
| train/                  |             |
|    approx_kl            | 0.014488839 |
|    clip_fraction        | 0.123       |
|    clip_range           | 0.2         |
|    entropy_loss         | -488        |
|  